In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sample").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [3]:
csvfile = spark.read.format("csv").option("inferSchema","true").option("header","true").load("C:/Users/b2en/Desktop/read write test/201508_trip_data.csv")

Q1. 데이터를 읽어서 parquet 형식 파일로 저장하시오

In [13]:
csvfile.write.format("parquet")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-parquet-file") #parquet 파일은 write 시에 header option을 안줘도 컬럼명을 읽어온다.

In [4]:
csvfile.write.format("csv")\
       .option("inferSchema","true").option("header","true")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-csv-file") #csv 저장

-----------------------------------------------------------------------------------------------------------------------

Q2. Start Station 별로 partition하여 저장하시오 (partition이 된다는 것은 안에 디렉토리로 구분이 된다는 것.)

In [5]:
csvfile.write.format("parquet")\
       .mode("overwrite")\
       .partitionBy("Start_Station")\
       .save("C:/Users/b2en/Desktop/read write test/my-parquet_file_start_station")

-----------------------------------------------------------------------------------------------------------------------

Q3. (non-partition으로) 데이터 셋을 1개의 parquet 파일로 저장하시오

In [6]:
csvfile.coalesce(1).write.format("parquet")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-parquet-file_withone")

In [7]:
csvfile.coalesce(1).write.format("csv")\
       .option("inferSchema","true").option("header","true")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-csv-file_withone")

-----------------------------------------------------------------------------------------------------------------------

Q4. (non-partition으로) 데이터 셋을 5개의 파일로 나누어 저장하시오

In [8]:
csvfile.repartition(5).write.format("parquet")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-parquet-file_withfive")

In [9]:
csvfile.repartition(5).write.format("csv")\
       .option("inferSchema","true").option("header","true")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-csv-file_withfive")

-----------------------------------------------------------------------------------------------------------------------

Q5. csv로 적재된 원본 데이터 셋을 partition 당 사이즈가 10MB를 넘지 않도록 나누어 data frame에 적재하시오.
  - partition은 spark의 partition으르 의미합니다.

In [16]:
spark.conf.set("spark.sql.files.maxPartitionBytes", 10000000)

In [17]:
limited_csvfile = spark.read.format("csv")\
                       .option("inferSchema","true").option("header","true")\
                       .load("C:/Users/b2en/Desktop/read write test/201508_trip_data.csv")

In [18]:
limited_csvfile.write.format("csv")\
               .option("inferSchema","true").option("header","true")\
               .mode("overwrite")\
               .save("C:/Users/b2en/Desktop/read write test/my-csv-file_limited")

-----------------------------------------------------------------------------------------------------------------------

Q6. (non-partition으로) 데이터 셋을 파일당 10만개 Row를 보관하도록 저장하시오.

In [26]:
csvfile.repartition(1).write.format("parquet")\
       .option("maxRecordsPerFile", 100000)\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-parquet-file_10M")#parquet는 컬럼명제외 10만개

In [23]:
csvfile.repartition(1).write.format("csv")\
       .option("maxRecordsPerFile", 100000)\
       .option("inferSchema","true").option("header","true")\
       .mode("overwrite")\
       .save("C:/Users/b2en/Desktop/read write test/my-csv-file_10M")#csv는 컬럼명 포함 10만1개

-----------------------------------------------------------------------------------------------------------------------

In [29]:
spark.read.format("csv")\
     .load("C:/Users/b2en/Desktop/read write test/my-csv-file_10M/part-00000-5c8d4f66-1644-4f3c-8020-cf9880b28b98-c002.csv").count()

100001

In [32]:
spark.read.format("parquet")\
     .load("C:/Users/b2en/Desktop/read write test/my-parquet-file_10M/part-00000-4f8c3057-78af-4b0e-9e94-bd467f6362a0-c002.snappy.parquet").count()

100000

In [7]:
spark.read.format("parquet")\
     .load("C:/Users/b2en/Desktop/read write test/my-parquet-file_10M/part-00000-f9ed8510-f4bd-40f7-bd3b-7dccc7178f73-c003.snappy.parquet").count()

54152

-----------------------------------------------------------------------------------------------------------------------

Q7. "Start Date" 컬럼을 기준으로 partition 형태로 저장하시오
      - 년도-월-일 별로 partition하여 저장하시오. (3개 partition key)
      - 단, 저장 시에 원본의 컬럼은 모두 유지되어야 함

In [21]:
from pyspark.sql.functions import *

splitcol = split(csvfile['Start_Date'], '/')


In [9]:
split_csvfile = \
csvfile.withColumn("year_time",splitcol.getItem(2))\
       .withColumn("month",splitcol.getItem(0))\
       .withColumn("day",splitcol.getItem(1))

In [10]:
split_space = split(split_csvfile["year_time"],' ')

In [11]:
processed_csvfile =\
split_csvfile.withColumn("year",split_space.getItem(0))\
             .drop("year_time")

In [13]:
processed_csvfile.write.format("parquet")\
                 .mode("overwrite")\
                 .partitionBy("year","month","day")\
                 .save("C:/Users/b2en/Desktop/read write test/my-parquet-file_date")

In [14]:
processed_csvfile.write.format("csv")\
                 .option("inferSchema","true").option("header","true")\
                 .mode("overwrite")\
                 .partitionBy("year","month","day")\
                 .save("C:/Users/b2en/Desktop/read write test/my-csv-file_date")

unix timestamp로 데이터프레임짜보기

In [31]:
csv_date =\
csvfile.withColumn("Start_Date2",from_unixtime(unix_timestamp(csvfile.Start_Date,"MM/dd/yyyy HH:mm")))\
       .withColumn("year",substring("Start_Date2",1,4))\
       .withColumn("month",substring("Start_Date2",6,2))\
       .withColumn("day",substring("Start_Date2",9,2))

In [32]:
csv_date.write.format("csv")\
                 .option("inferSchema","true").option("header","true")\
                 .mode("overwrite")\
                 .partitionBy("year","month","day")\
                 .save("C:/Users/b2en/Desktop/read write test/my-csv-file_date2")

-----------------------------------------------------------------------------------------------------------------------

Q7-a. partition으로 저장한 파일을 read 하시오.
      - 년도, 월, 일 조건 입력 시 partition prunning이 발생하는 지 확인하시오.
      - 저장한 파일을 df로 적재하여, 특정 조건만 추출하는 spark program 테스트를 진행합니다.
      - partition prunning의 발생 여부 확인 방법은 각자의 지식과 아이디어로 찾아주시고, 합리적으로 확인할 수만 있으면 가능합니다.

 

In [4]:
saved_pc_csv = spark.read.format("csv")\
                    .option("inferSchema","True").option("header","true").option("basepath","file:///C:/Users/b2en/Desktop/read write test/my-csv-file_date/")\
                    .load("C:/Users/b2en/Desktop/read write test/my-csv-file_date/year=2014/month=9/day=10/")

In [16]:
saved_pc_csv

Trip_ID,Duration,Start_Date,Start_Station,Start_Terminal,End_Date,End_Station,End_Terminal,Bike_#,Subscriber_Type,Zip_Code
447446,189,9/10/2014 23:16,San Jose Diridon ...,2,9/10/2014 23:19,Santa Clara at Al...,4,145,Subscriber,95110
447445,339,9/10/2014 23:09,San Francisco Cal...,69,9/10/2014 23:15,5th at Howard,57,524,Subscriber,94107
447443,264,9/10/2014 22:49,Howard at 2nd,63,9/10/2014 22:53,2nd at South Park,64,321,Subscriber,94107
447442,735,9/10/2014 22:48,Powell Street BART,39,9/10/2014 23:00,San Francisco Cal...,69,387,Subscriber,94107
447440,751,9/10/2014 22:30,2nd at Townsend,61,9/10/2014 22:42,Embarcadero at Sa...,60,481,Customer,37402
447439,770,9/10/2014 22:29,2nd at Townsend,61,9/10/2014 22:42,Embarcadero at Sa...,60,512,Customer,37402
447438,61,9/10/2014 22:24,2nd at Townsend,61,9/10/2014 22:25,2nd at Townsend,61,406,Subscriber,94107
447437,2329,9/10/2014 22:23,San Jose Civic Ce...,3,9/10/2014 23:02,San Jose Civic Ce...,3,156,Customer,63051
447436,709,9/10/2014 22:17,2nd at Townsend,61,9/10/2014 22:29,Broadway St at Ba...,82,334,Subscriber,94112
447435,1147,9/10/2014 22:14,University and Em...,35,9/10/2014 22:33,Park at Olive,38,21,Customer,94306


In [5]:
saved_pc_csv.explain(True)

== Parsed Logical Plan ==
Relation[Trip_ID#54,Duration#55,Start_Date#56,Start_Station#57,Start_Terminal#58,End_Date#59,End_Station#60,End_Terminal#61,Bike_##62,Subscriber_Type#63,Zip_Code#64] csv

== Analyzed Logical Plan ==
Trip_ID: int, Duration: int, Start_Date: string, Start_Station: string, Start_Terminal: int, End_Date: string, End_Station: string, End_Terminal: int, Bike_#: int, Subscriber_Type: string, Zip_Code: string
Relation[Trip_ID#54,Duration#55,Start_Date#56,Start_Station#57,Start_Terminal#58,End_Date#59,End_Station#60,End_Terminal#61,Bike_##62,Subscriber_Type#63,Zip_Code#64] csv

== Optimized Logical Plan ==
Relation[Trip_ID#54,Duration#55,Start_Date#56,Start_Station#57,Start_Terminal#58,End_Date#59,End_Station#60,End_Terminal#61,Bike_##62,Subscriber_Type#63,Zip_Code#64] csv

== Physical Plan ==
*(1) FileScan csv [Trip_ID#54,Duration#55,Start_Date#56,Start_Station#57,Start_Terminal#58,End_Date#59,End_Station#60,End_Terminal#61,Bike_##62,Subscriber_Type#63,Zip_Code#64] Ba

In [17]:
saved_pc_csv.count()

1351

In [6]:
saved_pc_csv_all = spark.read.format("csv")\
                        .option("inferSchema","True").option("header","true").option("basepath","C:/Users/b2en/Desktop/read write test/my-csv-file_date/")\
                        .load("C:/Users/b2en/Desktop/read write test/my-csv-file_date/")

In [ ]:
saved_pc_csv_all.where((saved_pc_csv_all["year"] == 2014) & (saved_pc_csv_all["month"] == 9) & (saved_pc_csv_all["day"] == 10))

In [6]:
saved_pc_csv_all.where((saved_pc_csv_all["year"] == 2014) & (saved_pc_csv_all["month"] == 9) & (saved_pc_csv_all["day"] == 10)).count()

1351

In [7]:
saved_pc_csv_all.where((saved_pc_csv_all["year"] == 2014) & (saved_pc_csv_all["month"] == 9) & (saved_pc_csv_all["day"] == 10)).explain(True)

== Parsed Logical Plan ==
Filter (((year#109 = 2014) && (month#110 = 9)) && (day#111 = 10))
+- Relation[Trip_ID#98,Duration#99,Start_Date#100,Start_Station#101,Start_Terminal#102,End_Date#103,End_Station#104,End_Terminal#105,Bike_##106,Subscriber_Type#107,Zip_Code#108,year#109,month#110,day#111] csv

== Analyzed Logical Plan ==
Trip_ID: int, Duration: int, Start_Date: string, Start_Station: string, Start_Terminal: int, End_Date: string, End_Station: string, End_Terminal: int, Bike_#: int, Subscriber_Type: string, Zip_Code: string, year: int, month: int, day: int
Filter (((year#109 = 2014) && (month#110 = 9)) && (day#111 = 10))
+- Relation[Trip_ID#98,Duration#99,Start_Date#100,Start_Station#101,Start_Terminal#102,End_Date#103,End_Station#104,End_Terminal#105,Bike_##106,Subscriber_Type#107,Zip_Code#108,year#109,month#110,day#111] csv

== Optimized Logical Plan ==
Filter (((((isnotnull(year#109) && isnotnull(day#111)) && isnotnull(month#110)) && (year#109 = 2014)) && (month#110 = 9)) && (